# An introduction to Elemental.jl

`Elemental.jl` is a Julia package that provides Julia bindings for the C++ library Elemental for distributed-memory dense and sparse-direct linear algebra and optimization which supports a wide range of functionality not available elsewhere. See more at http://libelemental.org/. The current wrappers support a subset of the functionality in Elemental but the process of adding functionality from Elemental is fairly simple.

The `Elemental.jl` provides three different APIs for Elemental. The first API tries to mimic the API of the C++ library as much as possible which makes it easy to use Elemental's focumentation while working with Elemental.jl. The second API extends some of Julia's linear algebra functions to work on Elemental's array types. Finally, there `Elemental.jl` defines linear algebra methods methods for `DArrays` where a conversion to and from Elemental's array types happens implicitly.

The library is based on MPI and hence an MPI installation is required before `Elemental.jl` can be installed. If the package `MPI.jl` is installed, it is possible to run MPI programs from within an interactive Julia session. In the following, we will show a few examples of using `Elemental.jl` interactively.

At first, we load `MPI.jl`, define an `MPIManager` with four workers, launch the workers with the `addprocs` function, and finally load `Elemental.jl`.

In [1]:
using MPI
man = MPIManager(np = 4)
addprocs(man)
using Elemental

We'll now use the `@mpi_do` macro to execute expressions on the MPI workers. We first intialize a distributed Elemental matrix with `Float64` elements.

In [2]:
@mpi_do man A = Elemental.DistMatrix(Float64);

In [3]:
@mpi_do man println(typeof(A))

	From worker 2:	Elemental.DistMatrix{Float64}
	From worker 3:	Elemental.DistMatrix{Float64}
	From worker 5:	Elemental.DistMatrix{Float64}
	From worker 4:	Elemental.DistMatrix{Float64}


Notice that we didn't specify the shape of the matrix. Following the C++ API, this can be done as part of the filling the matrix with values. We'll here fill it with Gaussian variates.

In [5]:
@mpi_do man Elemental.gaussian!(A, 4000, 2000);

In [6]:
@time @mpi_do man vals = svdvals(A);

  2.381227 seconds (1.35 k allocations: 86.720 KiB)


We can print the largest value. Notice that it will be printed on all workers.

In [7]:
@mpi_do man println(vals[1])

	From worker 3:	107.85799536311406
	From worker 5:	107.85799536311406
	From worker 4:	107.85799536311406
	From worker 2:	107.85799536311406


An alternative would be to work with `DArray` from the `DistributedArrays` package and have `Elemental.jl` do the conversion implicitly. That would instead look like

In [8]:
AD = DistributedArrays.drandn(4000, 2000)

4000×2000 DistributedArrays.DArray{Float64,2,Array{Float64,2}}:
  0.335499    0.318032    3.28741    0.592515   …  -1.31524     0.038773  
 -0.154335    0.280666    0.527864  -0.216495      -0.727533    0.126182  
  1.36767    -0.0761083   0.648303  -0.68143       -2.58291     0.922192  
  0.142799   -1.12469    -1.15906    0.849784       0.258645    0.329292  
 -1.31024    -1.28328     0.663565  -1.30362        1.51899     0.0446908 
  0.102125    0.0112773  -2.28842    0.0440335  …   1.55138    -0.301289  
 -0.137411   -0.0379651  -0.673978   0.213859       1.61906    -0.460501  
 -0.624762    0.130119   -1.22233   -1.50813        0.867158    0.424131  
  0.711469   -1.86969    -0.683314  -1.71519       -0.0936943  -0.917402  
 -0.0663845  -0.891755   -0.555031   0.0409641     -1.48966    -0.593472  
 -0.982027   -1.00371    -1.31893    1.29424    …  -0.527498    0.0630081 
  0.275746   -0.152106   -0.807856   1.7084         0.57542     2.10345   
 -0.510743   -0.574049    0.145004  

In [10]:
@time svdvals(AD)

  3.278759 seconds (2.00 k allocations: 144.203 KiB)


2000×1 DistributedArrays.DArray{Float64,2,Array{Float64,2}}:
 107.989 
 107.619 
 107.069 
 106.815 
 106.677 
 106.526 
 106.155 
 106.004 
 105.668 
 105.576 
 105.449 
 105.392 
 105.314 
   ⋮     
  19.8653
  19.7438
  19.6669
  19.6284
  19.58  
  19.4592
  19.4138
  19.2964
  19.0615
  18.9466
  18.817 
  18.6887

This has the advantage that `DArray` are more similar to normal Julia arrays and therefore convenient to work with. However, the data has to be copied back and forth between the two different ditributed array representations so there is some overhead associated with the convenience.